# _DeepFashion_ Autoencoder using Inception NetV3 extracted features

Inception NetV3 outputs (None, 2048) volume. To further reduce it to 90-dimensions, an autoencoder is used on the extracted features.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, InputLayer
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import RandomNormal, Constant

import matplotlib.pyplot as plt
import pickle, os, time

In [2]:
print(f"TensorFlow version: {tf.__version__}")

TensorFlow version: 2.5.0


In [3]:
num_gpus = len(tf.config.list_physical_devices('GPU'))
print(f"number of GPUs available = {num_gpus}")

number of GPUs available = 1


In [5]:
# Read pickled Python3 list containing 2048-d extracted feature representation per image-
features_list = pickle.load(open("DeepFashion_features_inceptionnetv3.pickle", "rb"))

In [6]:
# Convert from Python3 list to numpy array-
features_list_np = np.asarray(features_list)

features_list_np.shape

(289229, 2048)

In [7]:
del features_list

In [8]:
# Read pickled Python3 list containing abolute path and filenames-
filenames_list = pickle.load(open("DeepFashion_filenames_inceptionnetv3.pickle", "rb"))

In [9]:
features_list_np.shape[0], len(filenames_list)

(289229, 289229)

In [10]:
# Note that the absolute path contains Google colab path-
filenames_list[1]

'/content/img/1981_Graphic_Ringer_Tee/img_00000002.jpg'

### Autoencoder:

In [31]:
class Autoencoder(Model):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        self.encoder = Sequential([
            InputLayer(input_shape = (2048, )),
            Dense(
                units = 1024, activation = 'relu',
                kernel_initializer = tf.keras.initializers.glorot_normal(),
                use_bias = True, bias_initializer = RandomNormal(mean = 0.0, stddev = 0.05)
                 ),
            Dense(
                units = 512, activation = 'relu',
                kernel_initializer = tf.keras.initializers.glorot_normal(),
                use_bias = True, bias_initializer = RandomNormal(mean = 0.0, stddev = 0.05)
                 ),
            Dense(
                units = 256, activation = 'relu',
                kernel_initializer = tf.keras.initializers.glorot_normal(),
                use_bias = True, bias_initializer = RandomNormal(mean = 0.0, stddev = 0.05)
                 ),
            Dense(
                units = 90, activation = 'relu',
                kernel_initializer = tf.keras.initializers.glorot_normal(),
                use_bias = True, bias_initializer = RandomNormal(mean = 0.0, stddev = 0.05)
                 ),
        ])
    
        self.decoder = Sequential([
            Dense(
                units = 256, activation = 'relu',
                kernel_initializer = tf.keras.initializers.glorot_normal(),
                use_bias = True, bias_initializer = RandomNormal(mean = 0.0, stddev = 0.05)
                 ),
            Dense(
                units = 512, activation = 'relu',
                kernel_initializer = tf.keras.initializers.glorot_normal(),
                use_bias = True, bias_initializer = RandomNormal(mean = 0.0, stddev = 0.05)
                 ),
            Dense(
                units = 1024, activation = 'relu',
                kernel_initializer = tf.keras.initializers.glorot_normal(),
                use_bias = True, bias_initializer = RandomNormal(mean = 0.0, stddev = 0.05)
                 ),
            Dense(
                units = 2048, activation = 'sigmoid',
                use_bias = True, bias_initializer = RandomNormal(mean = 0.0, stddev = 0.05)
                 ),
        ])
    
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    

In [32]:
# Initialize and build defined autoencoder-
model = Autoencoder()
model.build(input_shape = (None, 2048))

In [33]:
model.summary()

Model: "autoencoder_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_4 (Sequential)    (None, 90)                2777434   
_________________________________________________________________
sequential_5 (Sequential)    (None, 2048)              2779392   
Total params: 5,556,826
Trainable params: 5,556,826
Non-trainable params: 0
_________________________________________________________________


In [34]:
tot_params = 0

for layer in model.trainable_weights:
    loc_param = tf.math.count_nonzero(layer, axis = None).numpy()
    tot_params += loc_param
    print(f"layer: {layer.shape} has {loc_param} wts")
    

layer: (2048, 1024) has 2097152 wts
layer: (1024,) has 1024 wts
layer: (1024, 512) has 524288 wts
layer: (512,) has 512 wts
layer: (512, 256) has 131072 wts
layer: (256,) has 256 wts
layer: (256, 90) has 23040 wts
layer: (90,) has 90 wts
layer: (90, 256) has 23040 wts
layer: (256,) has 256 wts
layer: (256, 512) has 131072 wts
layer: (512,) has 512 wts
layer: (512, 1024) has 524288 wts
layer: (1024,) has 1024 wts
layer: (1024, 2048) has 2097152 wts
layer: (2048,) has 2048 wts


In [35]:
print(f"Total number of trainable parameters in Autoencoder: {tot_params}")

Total number of trainable parameters in Autoencoder: 5556826


In [36]:
# Sanity check-
x = features_list_np[:5, :]
x.shape

(5, 2048)

In [37]:
x_encoded = model.encoder(x)
x_encoded.shape

TensorShape([5, 90])

In [38]:
x_reconstructed = model.decoder(x_encoded)
x_reconstructed.shape

TensorShape([5, 2048])

In [39]:
# Compile model-
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
    loss = tf.keras.losses.MeanSquaredError()
    )

In [ ]:
# Define early stopping criterion-
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss', min_delta = 0.001, patience = 3, verbose = 0,
    mode = 'auto', baseline = None, restore_best_weights = False
)

In [ ]:
# Train autoencoder-
history = model.fit(
    x = features_list_np, y = features_list_np,
    epochs = 30, batch_size = 32)

In [ ]:
history.history.keys()